# L7: Build an End-to-End Customer Service System Chatbot
- Step 1: Check the input with Moderation API.
- Step 2: Then extract the list of products.
- Step 3: If the product are found, we will look them up.
- Step 4: Answer the user question.
- Step 5: Put the answer thru the moderation API.

In [1]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
from collections import defaultdict
import json
import panel as pn # GUI
pn.extension()

_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [2]:
def get_completion_from_messages(messages, 
                                 model="gpt-3.5-turbo", 
                                 temperature=0, 
                                 max_tokens=500):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens, 
    )
    return response.choices[0].message["content"]

In [3]:
products_file = "products.json"
categories_file = "categories.json"

## Utils Functions Step 1: Check the input with Moderation API.

In [4]:
# Helper function for step 1
def get_products():
    with open(products_file, "r") as file:
        products = json.load(file)
    return products

def get_products_and_category():
    """
    Used in L5
    """
    products = get_products()
    products_by_category = defaultdict(list)
    for product_name, product_info in products.items():
        category = product_info.get("category")
        if category:
            products_by_category[category].append(product_info.get("name"))

    return dict(products_by_category)

def find_category_and_product_only(user_input, products_and_category):
    delimiter = "####"
    system_message = f"""
    You will be provided with customer service queries. \
    The customer service query will be delimited with \
    {delimiter} characters.
    Output a python list of objects, where each object has \
    the following format:
        'category': <one of Computers and Laptops, \
        Smartphones and Accessories, \
        Televisions and Home Theater Systems, \
        Gaming Consoles and Accessories, 
        Audio Equipment, Cameras and Camcorders>,
    OR
        'products': <a list of products that must \
        be found in the allowed products below>

    Where the categories and products must be found in \
    the customer service query.
    If a product is mentioned, it must be associated with \
    the correct category in the allowed products list below.
    If no products or categories are found, output an \
    empty list.
    If a category is mentioned, then provide the list of products.

    Allowed products: 

    Computers and Laptops category:
    TechPro Ultrabook
    BlueWave Gaming Laptop
    PowerLite Convertible
    TechPro Desktop
    BlueWave Chromebook

    Smartphones and Accessories category:
    SmartX ProPhone
    MobiTech PowerCase
    SmartX MiniPhone
    MobiTech Wireless Charger
    SmartX EarBuds

    Televisions and Home Theater Systems category:
    CineView 4K TV
    SoundMax Home Theater
    CineView 8K TV
    SoundMax Soundbar
    CineView OLED TV

    Gaming Consoles and Accessories category:
    GameSphere X
    ProGamer Controller
    GameSphere Y
    ProGamer Racing Wheel
    GameSphere VR Headset

    Audio Equipment category:
    AudioPhonic Noise-Canceling Headphones
    WaveSound Bluetooth Speaker
    AudioPhonic True Wireless Earbuds
    WaveSound Soundbar
    AudioPhonic Turntable

    Cameras and Camcorders category:
    FotoSnap DSLR Camera
    ActionCam 4K
    FotoSnap Mirrorless Camera
    ZoomMaster Camcorder
    FotoSnap Instant Camera

    Only output the list of objects, with nothing else.
    """
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": f"{delimiter}{user_input}{delimiter}"},
    ]
    return get_completion_from_messages(messages)


## Utils Functions Step 2: Then extract the list of products

In [5]:
def read_string_to_list(input_string):
    if input_string is None:
        return None

    try:
        # Ensure the input is a string
        if isinstance(input_string, list):
            input_string = str(input_string)
        
        input_string = input_string.replace(
            "'", '"'
        )  # Replace single quotes with double quotes for valid JSON
        data = json.loads(input_string)
        return data
    except json.JSONDecodeError:
        print("Error: Invalid JSON string")
        return None

## Utils Functions Step 3: If product is found, we will look them up

In [6]:
# product look up (either by category or by product within category)
def get_product_by_name(name):
    products = get_products()
    return products.get(name, None)

def get_products_by_category(category):
    products = get_products()
    return [product for product in products.values() if product["category"] == category]

def generate_output_string(data_list):
    output_string = ""

    if data_list is None:
        return output_string

    for data in data_list:
        try:
            if "products" in data:
                products_list = data["products"]
                for product_name in products_list:
                    product = get_product_by_name(product_name)
                    if product:
                        output_string += json.dumps(product, indent=4) + "\n"
                    else:
                        print(f"Error: Product '{product_name}' not found")
            elif "category" in data:
                category_name = data["category"]
                category_products = get_products_by_category(category_name)
                for product in category_products:
                    output_string += json.dumps(product, indent=4) + "\n"
            else:
                print("Error: Invalid object format")
        except Exception as e:
            print(f"Error: {e}")

    return output_string


## System of chained prompts for processing the user query

In [7]:
def process_user_message(user_input, all_messages, debug=True):
    delimiter = "```"
    
    # Step 1: Check input to see if it flags the Moderation API or is a prompt injection
    response = openai.Moderation.create(input=user_input)
    moderation_output = response["results"][0]

    if moderation_output["flagged"]:
        print("Step 1: Input flagged by Moderation API.")
        return "Sorry, we cannot process this request."

    if debug: print("Step 1: Input passed moderation check.")
    
    category_and_product_response = find_category_and_product_only(user_input, get_products_and_category())
    print(category_and_product_response)
    
    # Step 2: Extract the list of products
    category_and_product_list = read_string_to_list(category_and_product_response)
    print(category_and_product_list)

    if debug: print("Step 2: Extracted list of products.")

    # Step 3: If products are found, look them up
    product_information = generate_output_string(category_and_product_list)
    if debug: print("Step 3: Looked up product information.")

    # Step 4: Answer the user question
    system_message = f"""
    You are a customer service assistant for a large electronic store. \
    Respond in a friendly and helpful tone, with concise answers. \
    Make sure to ask the user relevant follow-up questions.
    """
    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': f"{delimiter}{user_input}{delimiter}"},
        {'role': 'assistant', 'content': f"Relevant product information:\n{product_information}"}
    ]

    final_response = get_completion_from_messages(all_messages + messages)
    if debug:print("Step 4: Generated response to user question.")
    all_messages = all_messages + messages[1:]

    # Step 5: Put the answer through the Moderation API
    response = openai.Moderation.create(input=final_response)
    moderation_output = response["results"][0]

    if moderation_output["flagged"]:
        if debug: print("Step 5: Response flagged by Moderation API.")
        return "Sorry, we cannot provide this information."

    if debug: print("Step 5: Response passed moderation check.")

    # Step 6: Ask the model if the response answers the initial user query well
    user_message = f"""
    Customer message: {delimiter}{user_input}{delimiter}
    Agent response: {delimiter}{final_response}{delimiter}

    Does the response sufficiently answer the question?
    """
    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': user_message}
    ]
    evaluation_response = get_completion_from_messages(messages)
    if debug: print("Step 6: Model evaluated the response.")

    # Step 7: If yes, use this answer; if not, say that you will connect the user to a human
    if "Y" in evaluation_response:  # Using "in" instead of "==" to be safer for model output variation (e.g., "Y." or "Yes")
        if debug: print("Step 7: Model approved the response.")
        return final_response, all_messages
    else:
        if debug: print("Step 7: Model disapproved the response.")
        neg_str = "I'm unable to provide the information you're looking for. I'll connect you with a human representative for further assistance."
        return neg_str, all_messages


In [8]:
user_input = "tell me about the smartx pro phone and the fotosnap camera, the dslr one. Also what tell me about your tvs"
response,_ = process_user_message(user_input,[])
print(response)

Step 1: Input passed moderation check.
[
    {'category': 'Smartphones and Accessories'},
    {'category': 'Cameras and Camcorders', 'products': ['FotoSnap DSLR Camera']},
    {'category': 'Televisions and Home Theater Systems'}
]
[{'category': 'Smartphones and Accessories'}, {'category': 'Cameras and Camcorders', 'products': ['FotoSnap DSLR Camera']}, {'category': 'Televisions and Home Theater Systems'}]
Step 2: Extracted list of products.
Step 3: Looked up product information.
Step 4: Generated response to user question.
Step 5: Response passed moderation check.
Step 6: Model evaluated the response.
Step 7: Model approved the response.
The SmartX ProPhone is a powerful smartphone with a 6.1-inch display, 128GB storage, 12MP dual camera, and 5G capability. It is priced at $899.99. The FotoSnap DSLR Camera features a 24.2MP sensor, 1080p video recording, 3-inch LCD screen, and interchangeable lenses, priced at $599.99.

As for TVs, we have a variety of options:
1. CineView 4K TV: 55-in

## Function that collects user and assistant messages over time

In [11]:
def collect_messages(debug=False):
    user_input = inp.value_input
    if debug: print(f"User Input = {user_input}")
    if user_input == "":
        return
    inp.value = ''
    global context
#     response, context = process_user_message(user_input, context, get_products_and_category(),debug=True)
    response, context = process_user_message(user_input, context, debug=False)
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(user_input, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)

## Chat with chatbot!
- Note that the system message includes details instructions about waht the OrderBot should do.

In [12]:
panels = [] # collect display 

context = [ {'role':'system', 'content':"You are Service Assistant"} ]  

inp = pn.widgets.TextInput( placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Service Assistant")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

Traceback (most recent call last):
 File "/home/mnguyen0226/anaconda3/lib/python3.9/site-packages/pyviz_comms/__init__.py", line 340, in _handle_msg
 self._on_msg(msg)
 File "/home/mnguyen0226/anaconda3/lib/python3.9/site-packages/panel/viewable.py", line 472, in _on_msg
 patch.apply_to_document(doc, comm.id if comm else None)
 File "/home/mnguyen0226/anaconda3/lib/python3.9/site-packages/bokeh/protocol/messages/patch_doc.py", line 104, in apply_to_document
 invoke_with_curdoc(doc, lambda: doc.apply_json_patch(self.payload, setter=setter))
 File "/home/mnguyen0226/anaconda3/lib/python3.9/site-packages/bokeh/document/callbacks.py", line 443, in invoke_with_curdoc
 return f()
 File "/home/mnguyen0226/anaconda3/lib/python3.9/site-packages/bokeh/protocol/messages/patch_doc.py", line 104, in <lambda>
 invoke_with_curdoc(doc, lambda: doc.apply_json_patch(self.payload, setter=setter))
 File "/home/mnguyen0226/anaconda3/lib/python3.9/site-packages/bokeh/document/document.py", line 391, in apply_json_patch
 DocumentPatchedEvent.handle_event(self, event, setter)
 File "/home/mnguyen0226/anaconda3/lib/python3.9/site-packages/bokeh/document/events.py", line 245, in handle_event
 event_cls._handle_event(doc, event)
 File "/home/mnguyen0226/anaconda3/lib/python3.9/site-packages/bokeh/document/events.py", line 280, in _handle_event
 cb(event.msg_data)
 File "/home/mnguyen0226/anaconda3/lib/python3.9/site-packages/bokeh/document/callbacks.py", line 390, in trigger_event
 model._trigger_event(event)
 File "/home/mnguyen0226/anaconda3/lib/python3.9/site-packages/bokeh/util/callback_manager.py", line 113, in _trigger_event
 self.document.callbacks.notify_event(cast(Model, self), event, invoke)
 File "/home/mnguyen0226/anaconda3/lib/python3.9/site-packages/bokeh/document/callbacks.py", line 260, in notify_event
 invoke_with_curdoc(doc, callback_invoker)
 File "/home/mnguyen0226/anaconda3/lib/python3.9/site-packages/bokeh/document/callbacks.py", line 443, in invoke_with_curdoc
 return f()
 File "/home/mnguyen0226/anaconda3/lib/python3.9/site-packages/bokeh/util/callback_manager.py", line 109, in invoke
 cast(EventCallbackWithEvent, callback)(event)
 File "/home/mnguyen0226/anaconda3/lib/python3.9/site-packages/panel/reactive.py", line 484, in _comm_event
 state._handle_exception(e)
 File "/home/mnguyen0226/anaconda3/lib/python3.9/site-packages/panel/io/state.py", line 458, in _handle_exception
 raise exception
 File "/home/mnguyen0226/anaconda3/lib/python3.9/site-packages/panel/reactive.py", line 482, in _comm_event
 self._process_bokeh_event(doc, event)
 File "/home/mnguyen0226/anaconda3/lib/python3.9/site-packages/panel/reactive.py", line 419, in _process_bokeh_event
 self._process_event(event)
 File "/home/mnguyen0226/anaconda3/lib/python3.9/site-packages/panel/widgets/button.py", line 241, in _process_event
 self.param.trigger('value')
 File "/home/mnguyen0226/anaconda3/lib/python3.9/site-packages/param/parameterized.py", line 2473, in trigger
 self_.update(dict(params, **triggers))
 File "/home/mnguyen0226/anaconda3/lib/python3.9/site-packages/param/parameterized.py", line 2318, in update
 restore = dict(self_._update(arg, **kwargs))
 File "/home/mnguyen0226/anaconda3/lib/python3.9/site-packages/param/parameterized.py", line 2351, in _update
 self_._batch_call_watchers()
 File "/home/mnguyen0226/anaconda3/lib/python3.9/site-packages/param/parameterized.py", line 2545, in _batch_call_watchers
 self_._execute_watcher(watcher, events)
 File "/home/mnguyen0226/anaconda3/lib/python3.9/site-packages/param/parameterized.py", line 2505, in _execute_watcher
 watcher.fn(*args, **kwargs)
 File "/home/mnguyen0226/anaconda3/lib/python3.9/site-packages/panel/param.py", line 882, in _replace_pane
 new_object = self.eval(self.object)
 File "/home/mnguyen0226/anaconda3/lib/python3.9/site-packages/panel/param.py", line 1097, in eval
 return eval_function_with_deps(ref)
 File "/home/mnguyen0226/anaconda3/lib/python3.9/site-packages/param/parameterized.py", line

Column
    [0] TextInput(placeholder='Enter text here…')
    [1] Row
        [0] Button(name='Service Assistant')
    [2] ParamFunction(function, _pane=Str, defer_load=False, height=300, loading_indicator=True)